Step-1:
Define input array X with angles from 60 deg to 360 deg converted in radians.

In [ ]:
import numpy as np


X_degree = np.arange(60, 361, 1)
X_rad = np.radians(X_degree)
X = X_rad.reshape(-1, 1)

X.shape


Step-2: Compute Y as Y=[Sin(X)+Cos(X)] + K^2
where K is a random number generated from normal distribution with 0.5 mean and
0.15 std dev.

In [ ]:
np.random.seed(42)
K = np.random.normal(loc=0.5, scale=0.15, size=X.shape[0])
Y = (np.sin(X.flatten()) + np.cos(X.flatten())) + K**2
Y = Y.reshape(-1, 1)

Step-3: Create Linear Regression model with SSE as error, and different Non-Linear
Regression models of 3, 13th and 15th degree polynomials on data created in step-1
and step-2. 

Step-4: Plot the created models for the power of 1, 3,13, 15, and print the SSE,
Coefficients for the plotted models, and print the prediction of test set on 15th Deg.
Polynomial model.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
def fit_poly_model(X_train, Y_train, X_test, Y_test, degree):
    poly = PolynomialFeatures(degree)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_poly, Y_train)
    Y_pred = model.predict(X_test_poly)
    sse = np.sum((Y_test - Y_pred)**2)
    
    return model, poly, sse, model.coef_


degrees = [1, 3, 13, 15]
models = {}

plt.figure(figsize=(14, 8))
X_plot = np.linspace(min(X.flatten()), max(X.flatten()), 300).reshape(-1, 1)

for degree in degrees:
    model, poly, sse, coefs = fit_poly_model(X_train, Y_train, X_test, Y_test, degree)
    Y_plot = model.predict(poly.transform(X_plot))
    
    plt.plot(X_plot, Y_plot, label=f'Degree {degree}')
    models[degree] = {
        'model': model,
        'poly': poly,
        'sse': sse,
        'coefs': coefs
    }
    print('\nDegree: ',degree)
    print('\nSSE: ',sse)
    print('Coefficients :',coefs)

# Prediction for 15th degree on test set
pred_15 = models[15]['model'].predict(models[15]['poly'].transform(X_test))
print('\nPrediction on 15th degree model (test set):')
print(pred_15.flatten())

plt.scatter(X, Y, color='black', label='Data')
plt.legend()
plt.title('Polynomial regression ')
plt.xlabel('X (radians)')
plt.ylabel('Y')
plt.grid(True)
plt.show()



Step-5: Add the L2 regularization to the nonlinear regression model with 15th degree
polynomial created in Step 3. (Optimize for Max. of 50 Iterations and lambda values
[1e-10 and 1e-5])

Step-6: Plot the Ridge (L2) regression models for lambda values [1e-10 and 1e-5] and
print the SSE, Coefficients for the plotted models, and print the prediction of test set
on 15th Deg. Polynomial model.

In [ ]:

lambdas = [1e-10, 1e-5]
degree = 15

plt.figure(figsize=(10, 6))
for lam in lambdas:
    poly = PolynomialFeatures(degree)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    X_plot_poly = poly.transform(X_plot)

    ridge_model = Ridge(alpha=lam, max_iter=50)
    ridge_model.fit(X_train_poly, Y_train)
    Y_plot_ridge = ridge_model.predict(X_plot_poly)
    
    sse_ridge = np.sum((Y_test - ridge_model.predict(X_test_poly))**2)
    
    plt.plot(X_plot, Y_plot_ridge, label=f'Ridge{lam}')
    print('\n ridge regression :',lam)
    print('\n SSE: ',sse_ridge)
    print('\n coefficients: ',ridge_model.coef_)
    print('\n prediction on test set:\n',ridge_model.predict(X_test_poly).flatten())

plt.scatter(X, Y, color='black', label='Data')
plt.legend()
plt.title('ridge regression L2 with 15th degree polynomial')
plt.xlabel('radians')
plt.ylabel('Y')
plt.grid(True)
plt.show()
